In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

zip_path = '/content/drive/My Drive/archive.zip'  # Replace with your ZIP file's path
destination_path = '/content/unzipped_files/'  # You can change the destination folder

# Unzipping the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(destination_path)

# List extracted files to confirm
!ls /content/unzipped_files/

demographic_info.txt  respiratory_sound_database  Respiratory_Sound_Database


In [ ]:
!pip install librosa
!pip install numpy
!pip install pandas
!pip install tensorflow
!pip install scikit-learn
!pip install matplotlib
!pip install seaborn

In [ ]:
from datetime import datetime
from os import listdir
from os.path import isfile, join

import librosa
import librosa.display

import numpy as np
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf

from sklearn.metrics import classification_report, confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn

In [ ]:
!pip install resampy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.0 MB/s eta 0:00:00


In [ ]:
# Define constants
max_pad_len = 862  # Desired length of Mel Spectrogram
c_names = ['Bronchiectasis', 'Bronchiolitis', 'COPD', 'Healthy', 'Pneumonia', 'URTI']
# Function to extract Mel spectrogram features
def extract_features(file_path, n_mels=128, max_len=300):
    y, sr = librosa.load(file_path, sr=None)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)

    # Pad or trim mel_spec_db to ensure consistent shape
    if mel_spec_db.shape[1] < max_len:
        # Pad with zeros if mel_spec_db is shorter than max_len
        pad_width = max_len - mel_spec_db.shape[1]
        mel_spec_db = np.pad(mel_spec_db, ((0, 0), (0, pad_width)), mode='constant')
    elif mel_spec_db.shape[1] > max_len:
        # Trim if mel_spec_db is longer than max_len
        mel_spec_db = mel_spec_db[:, :max_len]

    return mel_spec_db


In [ ]:
audio_path = '/content/unzipped_files/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/'
diagnosis_csv = '/content/unzipped_files/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv'
!ls '/content/unzipped_files/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files/'

101_1b1_Al_sc_Meditron.txt  150_1b2_Al_sc_Meditron.txt	178_1b2_Tc_mc_AKGC417L.txt
101_1b1_Al_sc_Meditron.wav  150_1b2_Al_sc_Meditron.wav	178_1b2_Tc_mc_AKGC417L.wav
101_1b1_Pr_sc_Meditron.txt  151_2p2_Al_mc_AKGC417L.txt	178_1b3_Al_mc_AKGC417L.txt
101_1b1_Pr_sc_Meditron.wav  151_2p2_Al_mc_AKGC417L.wav	178_1b3_Al_mc_AKGC417L.wav
102_1b1_Ar_sc_Meditron.txt  151_2p2_Ar_mc_AKGC417L.txt	178_1b3_Ar_mc_AKGC417L.txt
102_1b1_Ar_sc_Meditron.wav  151_2p2_Ar_mc_AKGC417L.wav	178_1b3_Ar_mc_AKGC417L.wav
103_2b2_Ar_mc_LittC2SE.txt  151_2p2_Ll_mc_AKGC417L.txt	178_1b3_Lr_mc_AKGC417L.txt
103_2b2_Ar_mc_LittC2SE.wav  151_2p2_Ll_mc_AKGC417L.wav	178_1b3_Lr_mc_AKGC417L.wav
104_1b1_Al_sc_Litt3200.txt  151_2p2_Lr_mc_AKGC417L.txt	178_1b3_Pl_mc_AKGC417L.txt
104_1b1_Al_sc_Litt3200.wav  151_2p2_Lr_mc_AKGC417L.wav	178_1b3_Pl_mc_AKGC417L.wav
104_1b1_Ar_sc_Litt3200.txt  151_2p2_Pl_mc_AKGC417L.txt	178_1b3_Pr_mc_AKGC417L.txt
104_1b1_Ar_sc_Litt3200.wav  151_2p2_Pl_mc_AKGC417L.wav	178_1b3_Pr_mc_AKGC417L.wav
104_1b1_Ll_sc_Li

In [ ]:
# Load the filenames and patient diagnosis
filenames = [f for f in listdir(audio_path) if (isfile(join(audio_path, f)) and f.endswith('.wav'))]
p_diag = pd.read_csv(diagnosis_csv, header=None)

!p_diag

/bin/bash: line 1: p_diag: command not found


In [ ]:
# Get patient IDs and their corresponding labels
p_id_in_file = [int(name[:3]) for name in filenames]
p_id_in_file = np.array(p_id_in_file)
labels = np.array([p_diag[p_diag[0] == x][1].values[0] for x in p_id_in_file])


In [ ]:
!pip install librosa
!pip install resampy


In [ ]:
# !pip uninstall librosa resampy
# !pip install librosa resampy


In [ ]:
# Define your parameters
max_len = 300  # Adjust this value as needed

# Extract features from all audio files
features = []
for file_name in filenames:
    file_path = join(audio_path, file_name)
    data = extract_features(file_path, max_len=max_len)
    if data is not None:
        features.append(data)

print(f'Finished feature extraction from {len(features)} files')

# Convert list to numpy array
features = np.array(features)

# Check the shape of the features
print(f'Features shape: {features.shape}')

Finished feature extraction from 920 files
Features shape: (920, 128, 300)


In [ ]:
# Filter out rare diseases (Asthma and LRTI)
features = np.delete(features, np.where((labels == 'Asthma') | (labels == 'LRTI'))[0], axis=0)
labels = np.delete(labels, np.where((labels == 'Asthma') | (labels == 'LRTI'))[0], axis=0)


In [ ]:
y = labels
features = np.expand_dims(features, axis=-1)
print(f'Features shape after adding channel dimension: {features.shape}')


Features shape after adding channel dimension: (917, 128, 300, 1)


In [ ]:
# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(features.shape[1], features.shape[2], features.shape[3])),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(y_categorical.shape[1], activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 298, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 149, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 147, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 73, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 71, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 35, 128)       0

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 298, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 149, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 147, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 73, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 71, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 35, 128)       0

In [ ]:
# Train the model
num_epochs = 250
num_batch_size = 128

callbacks = [
    ModelCheckpoint(
        filepath='mel_spectrogram_{epoch:02d}.keras',
        save_best_only=True,
        monitor='val_accuracy',
        verbose=1)
]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, y_categorical, test_size=0.2, random_state=42)

start = datetime.now()
# Train the model
history = model.fit(X_train, y_train, epochs=num_epochs, batch_size=num_batch_size, validation_data=(X_test, y_test))
duration = datetime.now() - start
print("Training completed in time: ", duration)

# Evaluate the model
score_train = model.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: ", score_train[1])

score_test = model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: ", score_test[1])

# Predictions
preds = model.predict(X_test)  # Predicted probabilities
class_preds = np.argmax(preds, axis=1)  # Predicted classes
y_testclass = np.argmax(y_test, axis=1)  # True classes

# Classification Report
c_names = ['Bronchiectasis', 'Bronchiolitis', 'COPD', 'Healthy', 'Pneumonia', 'URTI']
report = classification_report(y_testclass, class_preds, target_names=c_names, output_dict=True)

# Convert the classification report into a pandas DataFrame for easier manipulation
df_report = pd.DataFrame(report).transpose()

# Compute confusion matrix
conf_matrix = confusion_matrix(y_testclass, class_preds)

# Compute False Positives
false_positives = conf_matrix.sum(axis=0) - np.diag(conf_matrix)

# Ensure that df_report includes only the relevant classes
df_report = df_report.loc[c_names]

# Add False Positives to the report DataFrame
df_report['false_positives'] = false_positives

# Compute accuracy
accuracy = np.trace(conf_matrix) / np.sum(conf_matrix)
print(f"Overall Accuracy: {accuracy:.4f}")

# Compute overall precision
precision_per_class = df_report['precision']
overall_precision = np.mean(precision_per_class)
print(f"Overall Precision: {overall_precision:.4f}")

# Print the final report
print("Classification Report:")
print(df_report[['precision', 'recall', 'f1-score', 'support', 'false_positives']])



Epoch 1/250
6/6 [==============================] - 8s 1s/step - loss: 59.5051 - accuracy: 0.6085 - val_loss: 7.5313 - val_accuracy: 0.8478
Epoch 2/250
6/6 [==============================] - 6s 1s/step - loss: 3.9640 - accuracy: 0.7326 - val_loss: 1.0473 - val_accuracy: 0.8478
Epoch 3/250
6/6 [==============================] - 6s 1s/step - loss: 0.7606 - accuracy: 0.8690 - val_loss: 0.7476 - val_accuracy: 0.8478
Epoch 4/250
6/6 [==============================] - 6s 1s/step - loss: 0.5938 - accuracy: 0.8690 - val_loss: 0.6268 - val_accuracy: 0.8478
Epoch 5/250
6/6 [==============================] - 6s 1s/step - loss: 0.5838 - accuracy: 0.8690 - val_loss: 0.5944 - val_accuracy: 0.8478
Epoch 6/250
6/6 [==============================] - 6s 1s/step - loss: 0.5376 - accuracy: 0.8690 - val_loss: 0.5937 - val_accuracy: 0.8478
Epoch 7/250
6/6 [==============================] - 6s 1s/step - loss: 0.4957 - accuracy: 0.8690 - val_loss: 0.5420 - val_accuracy: 0.8478
Epoch 8/250
6/6 [================

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# Display model architecture summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 298, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 149, 32)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 147, 64)       18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 73, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 71, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 35, 128)       0